In [1]:
import numpy as np
import pandas as pd
import random
import time

import argparse

from rdkit import Chem
from rdkit.Chem import ChemicalFeatures
from rdkit import RDConfig
import networkx as nx

import shutil
import os
from os import listdir
from os.path import isfile, join
import xml.etree.ElementTree as ET

# Torch
import torch
import torch.optim as optim
import torch.nn as nn
from torch.autograd import Variable
from torch.autograd.variable import Variable
import torch.nn.functional as F

from __future__ import print_function

from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

import pickle
import copy
import datetime
import jsonpickle

from itertools import chain

In [3]:
from functions.SecondOrder import szymanski_ts_eq_fold
from functions.GraphReader import qm9_edges,qm9_nodes
from functions.GraphReader import get_values,get_graph_stats

from functions.Utlis import AverageMeter

from models.MPNN import MPNN

In [5]:
# Load data

In [6]:
dat_label = pd.read_csv('dataset.csv',index_col=0)
#label = label.iloc[:12,:]
dat_smiles = pd.read_csv('smiles.csv', index_col=0)
dict_smile = dat_smiles.to_dict()['SMILES']
# np.save('dict_smile.npy', dict_smile)

In [7]:
dat_labels = dat_label.T

In [8]:
from sklearn.utils.class_weight import compute_class_weight
lst_imbalance_ratio = []
for i in dat_label.values:
    class_weights=compute_class_weight('balanced',classes=np.unique(i), y=i)
    ratio = class_weights[1]
    lst_imbalance_ratio.append(ratio)

import math
weights = []
for i in range(len(lst_imbalance_ratio)):
    weights.append(math.log(1+lst_imbalance_ratio[i]))
weights = torch.tensor(weights)

In [9]:
#### split n-folds
#from sklearn.model_selection import KFold, StratifiedKFold
from scipy.sparse import lil_matrix
lil_matrix(dat_labels.values)

n_splits = 5

folds =szymanski_ts_eq_fold(n_splits, dat_labels)

In [10]:
# Parser check
def restricted_float(x, inter):
    x = float(x)
    if x < inter[0] or x > inter[1]:
        raise argparse.ArgumentTypeError("%r not in range [1e-5, 1e-4]"%(x,))
    return x

# Argument parser
import sys
sys.argv=['']
del sys

parser = argparse.ArgumentParser(description='Neural message passing')

#parser.add_argument('--dataset', default='qm9', help='QM9')
#parser.add_argument('--datasetPath', default='/home/taobai/Documents/Model/MPNN/mpnn-data/qm9/dsgdb9nsd/', help='dataset path')
#parser.add_argument('--logPath', default='./log/qm9/mpnn/', help='log path')
#parser.add_argument('--plotLr', default=False, help='allow plotting the data')
#parser.add_argument('--plotPath', default='./plot/qm9/mpnn/', help='plot path')
#parser.add_argument('--resume', default='./checkpoint/qm9/mpnn/',
#                    help='path to latest checkpoint')

# Optimization Options
parser.add_argument('--batch-size', type=int, default=10, metavar='N',
                    help='Input batch size for training (default: 20)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='Enables CUDA training')
parser.add_argument('--epochs', type=int, default=300, metavar='N',
                    help='Number of epochs to train (default: 100)')
parser.add_argument('--lr', type=lambda x: restricted_float(x, [1e-5, 1e-2]), default=1e-4, metavar='LR',
                    help='Initial learning rate [1e-5, 5e-4] (default: 1e-4)')
parser.add_argument('--lr-decay', type=lambda x: restricted_float(x, [.01, 1]), default=0.5, metavar='LR-DECAY',
                    help='Learning rate decay factor [.01, 1] (default: 0.6)')
parser.add_argument('--schedule', type=list, default=[0.1, 0.9], metavar='S',
                    help='Percentage of epochs to start the learning rate decay [0, 1] (default: [0.1, 0.9])')
parser.add_argument('--momentum', type=float, default=0.9, metavar='M',
                    help='SGD momentum (default: 0.9)')
# i/o
parser.add_argument('--log-interval', type=int, default=100, metavar='N',
                    help='How many batches to wait before logging training status')
# Accelerating
parser.add_argument('--prefetch', type=int, default=4, help='Pre-fetching threads.')

# Model modification
# parser.add_argument('--model', type=str,help='MPNN model name [MPNN, MPNNv2, MPNNv3]',default='MPNN')

_StoreAction(option_strings=['--prefetch'], dest='prefetch', nargs=None, const=None, default=4, type=<class 'int'>, choices=None, help='Pre-fetching threads.', metavar=None)

In [11]:
args = parser.parse_args()
print(args)

Namespace(batch_size=10, no_cuda=False, epochs=300, lr=0.0001, lr_decay=0.5, schedule=[0.1, 0.9], momentum=0.9, log_interval=100, prefetch=4)


In [12]:
args.cuda = not args.no_cuda and torch.cuda.is_available()

In [13]:
best_er1 = 0

In [14]:
def xyz_graph_reader(CID):
    dict_smile = np.load('dict_smile.npy',allow_pickle=True).item()
    smiles = dict_smile[CID]
    m = Chem.MolFromSmiles(smiles)
    m = Chem.AddHs(m)
   
    g = nx.Graph()
    # Create nodes
    for i in range(0, m.GetNumAtoms()):
        atom_i = m.GetAtomWithIdx(i)

        g.add_node(i, a_type=atom_i.GetSymbol(), a_num=atom_i.GetAtomicNum(), acceptor=0, donor=0,
                   aromatic=atom_i.GetIsAromatic(), hybridization=atom_i.GetHybridization(),
                   num_h=atom_i.GetTotalNumHs())


    # Read Edges
    for i in range(0, m.GetNumAtoms()):
        for j in range(0, m.GetNumAtoms()):
            e_ij = m.GetBondBetweenAtoms(i, j)
            if e_ij is not None:
                g.add_edge(i, j, b_type=e_ij.GetBondType())
            else:
                # Unbonded
                g.add_edge(i, j, b_type=None)

    l = list(dat_label[CID].values)           
                
    return g , l

In [15]:
class Qm9():
#class Qm9(data.Dataset):
    # Constructor
    def __init__(self, idx, vertex_transform=qm9_nodes, edge_transform=qm9_edges,
                 target_transform=None, e_representation='raw_distance'):
        self.idx = idx
        self.vertex_transform = vertex_transform
        self.edge_transform = edge_transform
        self.target_transform = target_transform
        self.e_representation = e_representation

    def __getitem__(self,index):
        
        g, target = xyz_graph_reader(self.idx[index])
        if self.vertex_transform is not None:
            h = self.vertex_transform(g)

        if self.edge_transform is not None:
            g, e = self.edge_transform(g)

        if self.target_transform is not None:
            target = self.target_transform(target)

        #g：adjacent matrix
        #h：node properties（list of list）
        #e：diction，key:edge，value:properties
        return (g, h, e), target

    def __len__(self):
        return len(self.idx)

    def set_target_transform(self, target_transform):
        self.target_transform = target_transform

In [16]:
# Data Loader

In [17]:
def collate_g(batch):

    batch_sizes = np.max(np.array([[len(input_b[1]), len(input_b[1][0]), len(input_b[2]),
                                len(list(input_b[2].values())[0])]
                                if input_b[2] else
                                [len(input_b[1]), len(input_b[1][0]), 0,0]
                                for (input_b, target_b) in batch]), axis=0)

    g = np.zeros((len(batch), batch_sizes[0], batch_sizes[0]))
    h = np.zeros((len(batch), batch_sizes[0], batch_sizes[1]))
    e = np.zeros((len(batch), batch_sizes[0], batch_sizes[0], batch_sizes[3]))

    target = np.zeros((len(batch), len(batch[0][1])))

    for i in range(len(batch)):

        num_nodes = len(batch[i][0][1])

        # Adjacency matrix
        g[i, 0:num_nodes, 0:num_nodes] = batch[i][0][0]

        # Node features
        h[i, 0:num_nodes, :] = batch[i][0][1]

        # Edges
        for edge in batch[i][0][2].keys():
            e[i, edge[0], edge[1], :] = batch[i][0][2][edge]
            e[i, edge[1], edge[0], :] = batch[i][0][2][edge]

        # Target
        target[i, :] = batch[i][1]

    g = torch.FloatTensor(g)
    h = torch.FloatTensor(h)
    e = torch.FloatTensor(e)
    target = torch.FloatTensor(target)

    return g, h, e, target

In [18]:
def train(train_loader, model, criterion, optimizer, epoch, logger=None):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    accuracy = AverageMeter()
    
    #list_loss = []

    # switch to train mode
    model.train()

    end = time.time()
    for i, (g, h, e, target) in enumerate(train_loader):

        # Prepare input data
        if args.cuda:
            g, h, e, target = g.cuda(), h.cuda(), e.cuda(), target.cuda()
        g, h, e, target = Variable(g), Variable(h), Variable(e), Variable(target)

        # Measure data loading time
        data_time.update(time.time() - end)

        optimizer.zero_grad()

        # Compute output
        output = model(g, h, e)
        train_loss = criterion(output, target)
        #list_loss.append(train_loss.item())
        # Logs
        losses.update(train_loss.item(), g.size(0))
        accuracy.update(evaluation(output, target).item(), g.size(0))
        
        # compute gradient and do SGD step
        train_loss.backward()
        optimizer.step()

        # Measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.log_interval == 0 and i > 0:

            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Accuracy {err.val:.4f} ({err.avg:.4f})'
                  .format(epoch, i, len(train_loader), batch_time=batch_time,
                          data_time=data_time, loss=losses, err=accuracy))

    print('Epoch: [{0}] Accuracy {err.avg:.3f}; Average Loss {loss.avg:.3f}; Avg Time x Batch {b_time.avg:.3f}'
          .format(epoch, err=accuracy, loss=losses, b_time=batch_time))
    
    #return list_loss

In [19]:
def validate(val_loader, model, criterion, evaluation, logger=None):
#     batch_time = AverageMeter()
    losses = AverageMeter()
    accuracy = AverageMeter()


    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (g, h, e, target) in enumerate(val_loader):

        # Prepare input data
        if args.cuda:
            g, h, e, target = g.cuda(), h.cuda(), e.cuda(), target.cuda()
        g, h, e, target = Variable(g), Variable(h), Variable(e), Variable(target)

        # Compute output
        output = model(g, h, e)

        # Logs
        losses.update(criterion(output, target).item(), g.size(0))
        accuracy.update(evaluation(output, target).item(), g.size(0))

#         # measure elapsed time
#         batch_time.update(time.time() - end)
#         end = time.time()

    print(' * Average accuracy {err.avg:.3f}; '
          .format(err=accuracy))


    return accuracy.avg

In [20]:
result_cl = []
result_auc = []
initial_lr = args.lr

In [ ]:
for n_folds in range(5):
    args.lr = initial_lr
    idxes = []
    for i in range(5):
        idxes.append(i)

    test_folds = int(n_folds)
    train_folds = idxes[:]
    train_folds.remove(n_folds)

    test_ids = [dat_label.columns[i] for i in folds[test_folds]]
    train_ids =  [dat_label.columns[i] for j in train_folds for i in folds[j]]

    data_train = Qm9(train_ids, edge_transform=qm9_edges, e_representation='raw_distance')
    data_test = Qm9(test_ids, edge_transform=qm9_edges, e_representation='raw_distance')

    # Select one graph
    g_tuple, l = data_train[0]
    g, h_t, e = g_tuple

    stat_dict = get_graph_stats(data_train[0])

    # set_target_transform was defined in Class Qm9
    data_train.set_target_transform(lambda x:x)
    data_test.set_target_transform(lambda x: x)
    
    
    # collate_g from utils
    train_loader = torch.utils.data.DataLoader(data_train,
                                           batch_size=args.batch_size,shuffle=True, collate_fn=collate_g,
                                           num_workers=args.prefetch, pin_memory=True)
    test_loader = torch.utils.data.DataLoader(data_test,
                                          batch_size=args.batch_size, collate_fn=collate_g,
                                          num_workers=args.prefetch, pin_memory=True)

    #点的特征维度，边的特征维度
    in_n = [len(h_t[0]), len(list(e.values())[0])] 
    #hidden state/embedding维度
    hidden_state_size = 20
    #邻居消息m_i维度（聚合后的维度）后面都用d_v表示
    message_size = 20
    #GNN层数
    n_layers = 3
    #labels数量
    l_target = len(l)
    #回归任务
    type ='classification'
    #type ='regression'

    #定义mpnn模型
    model = MPNN(in_n, hidden_state_size, message_size, n_layers, l_target, type=type)

    del in_n, hidden_state_size, message_size, n_layers, l_target, type

    #print('Optimizer')
    optimizer = torch.optim.AdamW(model.parameters(), lr=args.lr)


    #回归任务使用MSE 1ose
    criterion = torch.nn.CrossEntropyLoss(weight=weights,reduction='mean')

    #评估指标，|a-b|/|b|
    #evaluation = lambda output, target: torch.mean(torch.abs(output - target) / torch.abs(target))
    evaluation = lambda output, target: torch.eq(torch.round(output),target).float().mean()


    lr_step = (args.lr-args.lr*args.lr_decay)/(args.epochs*args.schedule[1] - args.epochs*args.schedule[0])

    print('Check cuda')
    if args.cuda:
        print('\t* Cuda')
        model = model.cuda()
        criterion = criterion.cuda()
        for state in optimizer.state.values():
            for k, v in state.items():
                if torch.is_tensor(v):
                    state[k] = v.cuda()


    # Epoch for loop
    for epoch in range(0, args.epochs):
        torch.cuda.empty_cache()

        if epoch > args.epochs * args.schedule[0] and epoch < args.epochs * args.schedule[1]:
            args.lr -= lr_step
            for param_group in optimizer.param_groups:
                param_group['lr'] = args.lr

        # train for one epoch
        # lst_loss = train(train_loader, model, criterion, optimizer, epoch)
        train(train_loader, model, criterion, optimizer, epoch)
        # evaluate on test set
        #er1 = validate(valid_loader, model, criterion, evaluation)

        #is_best = er1 > best_er1
        #best_er1 = min(er1, best_er1)

        
    # For testing
    report = []
    all_pred = []
    all_target =[]
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    accuracy = AverageMeter()

    if args.cuda:
        print('\t* Cuda')
        model = model.cuda()
        criterion = criterion.cuda()
    for i, (g, h, e, target) in enumerate(test_loader):
        torch.cuda.empty_cache()
        # Prepare input data
        if args.cuda:
            g, h, e, target = g.cuda(), h.cuda(), e.cuda(), target.cuda()
        g, h, e, target = Variable(g), Variable(h), Variable(e), Variable(target)

        # Compute output
        output =model(g, h, e)
        preds = torch.round(output)

        all_pred.append(preds.cpu().detach().numpy())
        all_target.append(target.cpu().detach().numpy())

        
        
    all_pred = np.vstack(all_pred)
    all_target = np.vstack(all_target)
    
    lst_auc = []
    for i in range(91):
        lst_auc.append(roc_auc_score(np.transpose(all_target)[i], np.transpose(all_pred)[i]))
    lst_auc.append(roc_auc_score(all_target, all_pred))

    cl = classification_report(all_target, all_pred,output_dict=True)
    report = pd.DataFrame.from_dict(cl, orient='index')
    
    result_cl.append(report)
    result_auc.append(lst_auc)
    
    del model,optimizer,g, h, e, target, output, preds, criterion
    torch.cuda.empty_cache()

In [ ]:
results_cl = result_cl[0]
for i in range(1,5):
    results_cl = results_cl + result_cl[i]

In [24]:
results_auc = (np.sum([i for i in result_auc], axis = 0))

In [25]:
results_cl.to_csv('12-15-01.csv')

In [ ]:
for i in results_auc:
    print(i)